# 📂 Gifty AI Worker: Category Matcher

Этот ноутбук сопоставляет категории сайтов-доноров с внутренней структурой категорий Gifty.
Можно запускать на CPU (Kaggle) или GPU для ускорения.

### Инструкция:
1. Добавьте свои данные в секцию **Configuration**.
2. Запустите все ячейки.

In [ ]:
!pip install -q requests transformers torch accelerate

In [ ]:
import requests
import time
import json
import torch
from transformers import pipeline
from kaggle_secrets import UserSecretsClient

class GiftyInternalClient:
    def __init__(self, api_base, token):
        self.api_base = api_base.rstrip("/")
        self.headers = {"X-Internal-Token": token}

    def get_category_tasks(self, limit=100):
        url = f"{self.api_base}/internal/categories/tasks?limit={limit}"
        resp = requests.get(url, headers=self.headers)
        resp.raise_for_status()
        return resp.json()

    def submit_categories(self, results):
        url = f"{self.api_base}/internal/categories/submit"
        resp = requests.post(url, json={"results": results}, headers=self.headers)
        resp.raise_for_status()
        return resp.json()

def extract_json(text):
    try:
        start = text.find('{')
        end = text.rfind('}') + 1
        return json.loads(text[start:end])
    except: 
        return None

## ⚙️ Configuration

In [ ]:
API_BASE = "https://api.giftyai.ru" # <-- ЗАМЕНИ

try:
    user_secrets = UserSecretsClient()
    API_TOKEN = user_secrets.get_secret("INTERNAL_API_TOKEN")
except:
    API_TOKEN = "your_secret_token_here"

# Список наших внутренних категорий (пример)
# В идеале, API должно отдавать этот список, но пока можно прописать тут
INTERNAL_CAT_MAP = [
    {"id": 1, "name": "Электроника и гаджеты"},
    {"id": 2, "name": "Дом и декор"},
    {"id": 3, "name": "Одежда и аксессуары"},
    {"id": 4, "name": "Хобби и творчество"},
    {"id": 5, "name": "Еда и напитки"},
    {"id": 6, "name": "Красота и уход"}
]

MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct" # Легкая модель для простой задачи

In [ ]:
print(f"Loading model {MODEL_NAME}...")
pipe = pipeline(
    "text-generation", 
    model=MODEL_NAME, 
    device_map="auto",
    model_kwargs={"torch_dtype": torch.bfloat16}
)

In [ ]:
def match_category(external_name):
    prompt = f"""Сопоставь внешнее название категории с наиболее подходящим ID из списка наших категорий.
    
    Список категорий Gifty:
    {json.dumps(INTERNAL_CAT_MAP, ensure_ascii=False)}
    
    Внешнее название: "{external_name}"
    
    Верни ТОЛЬКО JSON формат:
    {{"internal_category_id": ID_ЧИСЛО}}"""
    
    messages = [{"role": "user", "content": prompt}]
    response = pipe(messages, max_new_tokens=64, return_full_text=False)[0]['generated_text']
    
    data = extract_json(response)
    return data.get('internal_category_id') if data else None

client = GiftyInternalClient(API_BASE, API_TOKEN)
print("Category Matcher started...")

while True:
    try:
        tasks = client.get_category_tasks(limit=50)
        if not tasks:
            print("No categories to match. Waiting 5 min...")
            time.sleep(300)
            continue
            
        results = []
        for task in tasks:
            int_id = match_category(task['external_name'])
            if int_id:
                results.append({
                    "external_name": task['external_name'],
                    "internal_category_id": int_id
                })
        
        if results:
            client.submit_categories(results)
            print(f"Matched {len(results)} categories.")
            
    except Exception as e:
        print(f"Error: {e}")
        time.sleep(10)